## Convert nd2 to tiff - OmniSegger

In [5]:
!pip install aicsimageio[nd2] bioformats_jar

   ---------------------------------------- 0.0/81.1 kB ? eta -:--:--
   -------------------- ------------------- 41.0/81.1 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 81.1/81.1 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/351.5 kB ? eta -:--:--
   -------- ------------------------------- 71.7/351.5 kB 3.8 MB/s eta 0:00:01
   ----------------------- ---------------- 204.8/351.5 kB 2.5 MB/s eta 0:00:01
   ---------------------------------------- 351.5/351.5 kB 3.1 MB/s eta 0:00:00


In [60]:
from aicsimageio import AICSImage
import platform

# open the nd2 file

input_path = "C:\\Users\\teres\\Downloads\\200310_491E_5min_CiT594_002.nd2"

if platform.system() == "Windows":
    input_path = r"{}".format(input_path)

imgfile = AICSImage(input_path)



In [34]:
# detect number of channels, time points, xy, z stacks

dims = imgfile.dims  # TCZYX

numt = dims['T']
numz = dims['Z']
# numxy = dims['Y']
numxy = len(imgfile.scenes)
numc = dims['C']
imsize = imgfile.shape[-2:]

print('timepoints:', numt[0], '\nz-planes:', numz[0],  '\nxy:', numxy,  '\nchannels:', numc[0], '\nimage size:', imsize)


timepoints: 1 
z-planes: 1 
xy: 1 
channels: 3 
image size: (2160, 2560)


In [61]:
# save as a tiff stack
import os

imgfile.save(f"{os.path.splitext(input_path)[0]}.tiff")
# aicsimageio.writers.ome_tiff_writer(img, 'imgname.tif')



In [58]:
# save metadata

import os

md = imgfile.metadata
with open(f"{os.path.splitext(input_path)[0]}_metadata.txt", "w") as f:
    f.write(str(md))
f.close()

In [51]:
from aicsimageio.writers import OmeTiffWriter
import os

# loop through each dim and save as individual tiff files
# basename_t1xy1c1.tif naming format for OmniSegger

# img  = imgfile.dask_data  # returns 5D TCZYX dask array

# Pull only a specific chunk in-memory
# lazy_t0 = img.get_image_dask_data("CZYX", T=0)  # returns out-of-memory 4D dask array
# t0 = lazy_t0.compute()  # returns in-memory 4D numpy array

def itersavetiff(filepath, numt, numxy, numc, numz):
    imgfile = AICSImage(filepath)
    basename = os.path.splitext(filepath)[0]
    if numz > 1:
        for zz in range(numz):
            zdir = os.path.join(os.path.dirname(filepath), f'z{zz}')
            if not os.path.exists(zdir):
                os.makedirs(zdir)
            for xy in range(numxy):
                imgfile.set_scene(xy)
                for tt in range(numt):
                    for cc in range(numc):
                        # singimg = imgdata[tt, c, zz, :, :]
                        singimg = imgfile.get_image_dask_data("YX", T=tt, C=cc, Z=zz).compute()
                        # save as tif, with t and c starting at 1 for OmniSegger
                        OmeTiffWriter.save(singimg,os.path.join(zdir, f'{basename}_t{tt+1}xy{xy}c{cc+1}.tif'))
    else:
        for xy in range(numxy):
            imgfile.set_scene(xy)
            for tt in range(numt):
                for cc in range(numc):
                    # singimg = imgdata[tt, c, 0, :, :]
                    singimg = imgfile.get_image_dask_data("YX", T=tt, C=cc).compute()
                    # save as tif, with t and c starting at 1 for OmniSegger
                    OmeTiffWriter.save(singimg,os.path.join(os.path.dirname(filepath), f'{basename}_t{tt+1}xy{xy}c{cc+1}.tif'))

# Example usage
# itersavetiff(input_path, img, numt[0], numxy, numc[0], numz[0])

In [52]:
itersavetiff(input_path, imgfile, numt[0], numxy, numc[0], numz[0])